In [1]:
import pymysql
import copy
import json

In [6]:
connection = pymysql.connect("127.0.0.1", "root", "root", "code-mapper-beta", cursorclass=pymysql.cursors.DictCursor)

# Upgrade

In [7]:
def upgrade(state):
    state = copy.deepcopy(state)
    
    if "cuiAssignment" not in state:
        print(" - Create cuiAssignment")
        state['cuiAssignment'] = {}
        cuis = [concept['cui'] for concept in state['mapping']['concepts']]
        for concept in state['indexing']['concepts']:
            include = concept['cui'] in cuis
            state['cuiAssignment'][concept['cui']] = \
                "include" if include else "exclude"
                
    if 'codingSystems' not in state:
        print(" - Move codingSystems")
        assert state['mapping']['codingSystems']
        state['codingSystems'] = state['mapping']['codingSystems']
        del state['mapping']['codingSystems']
        
    if 'semanticTypes' in state['mapping']:
        print(" - Remove semanticTypes")
        del state['mapping']['semanticTypes']
        
    if any('comments' not in concept for concept in state['mapping']['concepts']):
        for concept in state['mapping']['concepts']:
            if 'comments' not in concept:
                concept['comments'] = []        
    
    if any('codingSystem' not in sourceConcept
           for concept in state['mapping']['concepts']
           for sourceConcept in concept['sourceConcepts']):
        print(" - Upgrade codingSystem fields")
        for concept in state['mapping']['concepts']:
            for source_concept in concept['sourceConcepts']:
                if 'vocabulary' in source_concept:
                    source_concept['codingSystem'] = source_concept['vocabulary']
                    del source_concept['vocabulary']
            for coding_system in concept['codes']:
                for source_concept in concept['codes'][coding_system]:
                    if 'vocabulary' in source_concept:
                        source_concept['codingSystem'] = source_concept['vocabulary']
                        del source_concept['vocabulary']
    return state

In [8]:
query = """
    SELECT id, name, state FROM `case_definitions`
"""
update = """
    UPDATE `case_definitions` SET `state` = %s WHERE `id` = %s
"""
with connection.cursor() as cursor:
    cursor.execute(query)
    for case_definition in cursor.fetchall():
        print('#', case_definition['name'])
        state = json.loads(case_definition['state'])
        new_state = json.dumps(upgrade(state))
        cursor.execute(update, (new_state, case_definition['id']))
        connection.commit()

# Pertussis BB
# test BB
# Infectious disease KG
# test rh
# Dementia
# thrombocytopenia BC
# thrombocytopenia PLAIN
# IHD_pmc
# pertussis PLAIN
# pertussis ECDC
# anaphylaxis PLAIN
# Bullous eruption_pmc
# Intussusception_pmc
# Thromboembolic events_pmc
# Rhabdomyolysis_pmc
# test again)IHD_pmc
# test2
# anaphylaxis BC
# anaphylaxis US-IAID
# gbs PLAIN
# gbs BC
# POC_pertussis
# pertussis v2
# aeaea
# injection_site_recation
# injection_site_recation-exclusion
# fever
# somnolence
# crying
# febrile seizure - convulsions
# generalized seizure - convulsions
# Hypotonic-hyporesponsive episode
# pneumonia


# Validate

In [16]:
import jsonschema
import yaml

schema = yaml.load(open('state-schema.yaml'))

def validate(state):
    jsonschema.validate(state, schema)

In [17]:
query = """
    SELECT id, name, state FROM `case_definitions`
"""
with connection.cursor() as cursor:
    cursor.execute(query)
    for case_definition in cursor.fetchall():
        print('#', case_definition['name'])
        state = json.loads(case_definition['state'])
        validate(state)

# Pertussis BB
# test BB
# Infectious disease KG
# test rh
# Dementia
# thrombocytopenia BC
# thrombocytopenia PLAIN
# IHD_pmc
# pertussis PLAIN
# pertussis ECDC
# anaphylaxis PLAIN
# Bullous eruption_pmc
# Intussusception_pmc
# Thromboembolic events_pmc
# Rhabdomyolysis_pmc
# test again)IHD_pmc
# test2
# anaphylaxis BC
# anaphylaxis US-IAID
# gbs PLAIN
# gbs BC
# POC_pertussis
# pertussis v2
# aeaea
# injection_site_recation
# injection_site_recation-exclusion
# fever
# somnolence
# crying
# febrile seizure - convulsions
# generalized seizure - convulsions
# Hypotonic-hyporesponsive episode
# pneumonia


ValidationError: {'argument': '', 'operation': 'Change coding systems', 'user': 'd.weibel', 'result': 'ICD10CM, ICD9CM, ICPC2EENG, ICPC2P, RCD, RCD2', 'date': '2015-11-24T12:51:52.538Z'} is not valid under any of the given schemas

Failed validating 'oneOf' in schema['properties']['mapping']['properties']['history']['items']:
    {'oneOf': [{'additionalProperties': False,
                'properties': {'argument': {'type': 'null'},
                               'date': {'format': 'date-time',
                                        'type': 'string'},
                               'operation': {'enum': ['Automatic '
                                                      'coding'],
                                             'type': 'string'},
                               'result': {'$ref': '#/definitions/reduced_concept_array'},
                               'user': {'type': 'string'}},
                'required': ['date',
                             'user',
                      